# Unione dati sensori HMOG

Il dataset HMOG è formato da 100 cartelle rappresentanti gli utenti. Per ogni utente, troviamo 24 sessioni di acquisizione dati suddivise in diversi file. 
Andiamo ora ad unire i file contenenti i dati di Accelerometro, Giroscopio e Magnetometro di ogni sessione.

Si inizia con l'importazione delle librerie.

In [ ]:
import os
from os.path import isdir, join
from os import listdir, path
import pandas as pd

La funzione di seguito serve a controllare che dopo l'unione dei file dei sensori, ogni riga contenga gli stessi dati di IdAttività, GestureScenario e PhoneOrientation

In [ ]:
def filterRowActivityScenarioOrientation(row):
    return (row["ActivityIDAcc"] == row["ActivityIDGyro"] == row["ActivityIDMagn"] and
            row["GestureScenarioAcc"] == row["GestureScenarioGyro"] == row["GestureScenarioMagn"] and
            row["PhoneOrientationAcc"] == row["PhoneOrientationGyro"] == row["PhoneOrientationMagn"])

Impostiamo il numero di cartelle che vogliamo prendere in considerazione e calcoliamo il totale di sessioni da analizzare in modo da avere un feedback durante l'esecuzione

In [ ]:
firstNFolder = 100
total=firstNFolder*24

Vengono specificati i nomi delle colonne per i vari file csv da analizzare:
<ul>
    <li>Activity.csv: contiene i dati per identificare il tipo di acquisizione (GestureScenario, TaskId, ecc...)</li>
    <li>Acceletometer.csv: contiene i dati rilevati dal sensore dell'accelerometro durante le acquisizioni</li>
    <li>Gyroscope.csv: contiene i dati rilevati dal sensore del giroscopio durante le acquisizioni</li>
    <li>Magnetometer.csv: contiene i dati rilevati dal sensore magnetometro durante le acquisizioni</li>
</ul>

In [ ]:
activityColumns = ["ActivityID", "SubjectID", "SessionNumber", "StartTime", "EndTime", "RelativeStartTime",
                   "RelativeEndTime", "GestureScenario", "TaskID", "ContentID"]
columnExtractAcc = ["ActivityIDAcc", "SubjectIDAcc", "SysTimeAcc", "GestureScenarioAcc", "XAcc", "YAcc", "ZAcc",
                    "PhoneOrientationAcc"]
columnExtractGyro = ["ActivityIDGyro", "SubjectIDGyro", "SysTimeGyro", "GestureScenarioGyro", "XGyro", "YGyro", "ZGyro",
                     "PhoneOrientationGyro"]
columnExtractMagn = ["ActivityIDMagn", "SubjectIDMagn", "SysTimeMagn", "GestureScenarioMagn", "XMagn", "YMagn", "ZMagn",
                     "PhoneOrientationMagn"]

Da notare che i file dei sensori hanno le stesse colonne. Viene utilizzato l'array sensorsColumns per identificarle durante il caricamento. Per il join tra i tre file vengono però rinominati per identificarli univocamente.
columnExtract viene utilizzato per estrarre le colonne del join tra Activity.csv e uno dei sensori.

In [ ]:
sensorsColumns = ["SysTime", "EventTime", "ActivityID", "X", "Y", "Z", "PhoneOrientation"]
columnExtract = ["ActivityID", "SubjectID", "SysTime", "GestureScenario", "X", "Y", "Z", "PhoneOrientation"]

Viene dichiarata la cartella contenente i file e i nomi dei vari file utilizzati. 
joinedFile sarà l'output file dello script.
Nel caso in cui il file sia già presente nella cartella viene eliminato.

In [ ]:
mainPath = "/home/tullio/Projects/fvab_dataset/public_dataset"
joinedFile = "JoinSensors.csv"
accelerometerName = "Accelerometer.csv"
gyroscopeName = "Gyroscope.csv"
magnetometerName = "Magnetometer.csv"
joinedFilePath = join(mainPath, joinedFile)
if path.exists(joinedFilePath): os.remove(joinedFilePath)

Si vanno ad identificare le cartelle degli utenti nella directory
printHead è una variabile che indica quanto stampare i nomi di colonne nel csv di output.

In [ ]:
subjectDirectories = [f for f in listdir(mainPath) if isdir(join(mainPath, f))]
subjectDirectories.sort()
printHead = True

Vengono scansionate e rinominate le cartelle delle sessioni in modo da mantenere un'ordinamento letterale.

In [ ]:
for dir in subjectDirectories:
    subjectDirectory = join(mainPath, dir)
    sessionDirectories = [f for f in listdir(subjectDirectory) if isdir(join(subjectDirectory, f))]
    for sessionDirectory in sessionDirectories:
        splitted = sessionDirectory.split('_')
        splitted[2] = splitted[2].rjust(2, '0')
        newName = ('_'.join(splitted))
        os.rename(join(subjectDirectory, sessionDirectory), join(subjectDirectory, newName))

Vengono quindi scansionati i file di ogni sessione (for interno). 
Viene effettuato il join tra il file ActivityCsv e i file dei sensori.
Viene identificato il join con il minor numero di righe e vengono tagliati gli altri.
Viene poi eseguita la funzione "filterRowActivityScenarioOrientation" su ogni riga per controllare che i valori di AcitivityId, GestureScenario e PhoneOrientation siano gli stessi per ogni riga.
Il risultato viene inserito nel file risultante.

In [ ]:
for dir in subjectDirectories:
    if (firstNFolder <= 0):
        break
    firstNFolder -= 1
    subjectDirectory = join(mainPath, dir)
    sessionDirectories = [f for f in listdir(subjectDirectory) if isdir(join(subjectDirectory, f))]
    sessionDirectories.sort()
    for sessionDirectory in sessionDirectories:
        path = join(subjectDirectory, sessionDirectory)
        accelerometerFile = join(path, accelerometerName)
        gyroscopeFile = join(path, gyroscopeName)
        magnetometerFile = join(path, magnetometerName)
        activityCsv = pd.read_csv(join(path, activityName), header=None, names=activityColumns)
        accelerometerCsv = pd.read_csv(accelerometerFile, header=None, names=sensorsColumns)
        gyroscopeCsv = pd.read_csv(gyroscopeFile, header=None, names=sensorsColumns)
        magnetometerCsv = pd.read_csv(magnetometerFile, header=None, names=sensorsColumns)
        joined = accelerometerCsv.merge(activityCsv, on="ActivityID")
        dfAcc = joined[columnExtract]
        dfAcc.columns = columnExtractAcc
        joined = gyroscopeCsv.merge(activityCsv, on="ActivityID")
        dfGyro = joined[columnExtract]
        dfGyro.columns = columnExtractGyro
        joined = magnetometerCsv.merge(activityCsv, on="ActivityID")
        dfMagn = joined[columnExtract]
        dfMagn.columns = columnExtractMagn
        maxLen = min(dfAcc.shape[0], dfGyro.shape[0], dfMagn.shape[0])
        result = pd.concat([dfAcc.head(maxLen), dfGyro.head(maxLen), dfMagn.head(maxLen)], axis=1, sort=False)
        result = result[result.apply(filterRowActivityScenarioOrientation, axis=1)]
        result = result[[
            "GestureScenarioAcc",
            "XAcc", "YAcc", "ZAcc",
            "XGyro", "YGyro", "ZGyro",
            "XMagn", "YMagn", "ZMagn"]]
        result.columns = [
            "GestureScenario",
            "XAcc", "YAcc", "ZAcc",
            "XGyro", "YGyro", "ZGyro",
            "XMagn", "YMagn", "ZMagn"]
        result.to_csv(joinedFilePath, mode='a', header=printHead, index=False)
        printHead = False
        total-=1
        print(total)

# Attenzione
L'esecuzione dello script richiede diverso tempo e produce in output un file di circa 13GB